In [9]:
# Ячейка 2: Импорты
import re
import csv
from datetime import datetime, timedelta
import subprocess
import os

In [10]:
# Ячейка 3: Время начала видео из имени файла
def get_video_start_time_from_filename(filename):
    # Пример: 20250726_131943.mp4
    match = re.search(r'(\d{8})_(\d{6})', filename)
    if not match:
        raise ValueError('Не удалось извлечь время из имени файла')
    date_part, time_part = match.groups()
    dt_str = f'{date_part} {time_part}'
    return datetime.strptime(dt_str, '%Y%m%d %H%M%S')

In [11]:
# Ячейка 4: Длительность видео через ffprobe
def get_video_duration(filename):
    # ffprobe должен быть установлен в системе
    result = subprocess.run([
        'ffprobe', '-v', 'error', '-show_entries', 'format=duration',
        '-of', 'default=noprint_wrappers=1:nokey=1', filename
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        raise RuntimeError(f'ffprobe error: {result.stderr}')
    return float(result.stdout.strip())

In [12]:
# Ячейка 5: Чтение точек из CSV
def read_points_from_csv(csv_path):
    points = []
    with open(csv_path, encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) < 7:
                continue
            try:
                point_num = int(row[0])
                start_time = datetime.strptime(row[5], '%Y-%m-%d %H:%M:%S.%f')
                end_time = datetime.strptime(row[6], '%Y-%m-%d %H:%M:%S.%f')
                points.append({'num': point_num, 'start': start_time, 'end': end_time})
            except Exception as e:
                # Пробуем без миллисекунд
                try:
                    start_time = datetime.strptime(row[5], '%Y-%m-%d %H:%M:%S')
                    end_time = datetime.strptime(row[6], '%Y-%m-%d %H:%M:%S')
                    points.append({'num': point_num, 'start': start_time, 'end': end_time})
                except Exception:
                    continue
    return points

In [13]:
# Ячейка 6: Генерация SRT
def timedelta_to_srt(tdelta):
    total_seconds = int(tdelta.total_seconds())
    ms = int((tdelta.total_seconds() - total_seconds) * 1000)
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f'{hours:02}:{minutes:02}:{seconds:02},{ms:03}'

def generate_srt(points, video_start, video_duration, srt_path):
    with open(srt_path, 'w', encoding='utf-8') as f:
        for idx, pt in enumerate(points, 1):
            # Сдвиг времени точки относительно старта видео
            rel_start = pt['start'] - video_start
            rel_end = pt['end'] - video_start
            # Ограничиваем в пределах видео
            if rel_end.total_seconds() < 0 or rel_start.total_seconds() > video_duration:
                continue
            rel_start = max(rel_start, timedelta(0))
            rel_end = min(rel_end, timedelta(seconds=video_duration))
            f.write(f'{idx}\n')
            f.write(f'{timedelta_to_srt(rel_start)} --> {timedelta_to_srt(rel_end)}\n')
            f.write(f'Точка {pt['num']}\n\n')

In [21]:
# Ячейка 7: Пример использования
video_file = r"C:\geowork\2025\Новополоцк\122.25 Парк культуры (Аллея)\Исходные\Фото, видео-абрисы\20250726_124748.mp4"  # имя файла
csv_file = r"C:\geowork\2025\2025-07-26 122-25 парк.csv"  # путь к csv
srt_file = r'C:\geowork\2025\Новополоцк\122.25 Парк культуры (Аллея)\Исходные\Фото, видео-абрисы\20250726_124748.srt'

video_start = get_video_start_time_from_filename(video_file)
video_duration = get_video_duration(video_file)
points = read_points_from_csv(csv_file)
generate_srt(points, video_start, video_duration, srt_file)
print(f'SRT-файл создан: {srt_file}')

SRT-файл создан: C:\geowork\2025\Новополоцк\122.25 Парк культуры (Аллея)\Исходные\Фото, видео-абрисы\20250726_124748.srt
